## Spectral sequence of a ring

In [1]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'
data_ring = plot.load_ss_ring(path_ring)
plot.export_ss_ring(data_ring)

## Spectral sequence of a module

In [1]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\Ceta_AdamsSS_t100.db'
data_ring, data_mod = plot.load_ss_mod(path_ring, path_mod)
plot.export_ss_mod(data_ring, data_mod)

## Homotopy of a ring

In [1]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\main\S0_AdamsSS_t261.db'
data = plot.load_pi_ring(path_ring)
plot.export_pi_ring(data)

## Homotopy of a module

In [70]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\C2_AdamsSS_t100.db'
data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
plot.export_pi_mod(data_ring, data_mod)

## Homotopy exact sequence

In [3]:
import plot
path_ring = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\S0_AdamsSS_t100.db'
path_mod = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\debug\Csigma_AdamsSS_t100.db'
data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
plot.export_pi_exact(data_ring, data_mod)

## Spectral sequence of a module from resolution database

In [2]:
import plot
path = R'C:\Users\lwnpk\Documents\Projects\algtop_cpp_build\bin\Release\X2_Adams_res.db'
data = plot.load_ss_from_res(path)
plot.export_ss_from_res(data)

## Update website pages

In [12]:
import plot
complexes = ["S0", "C2", "Ceta", "Cnu", "Csigma"]

dir_db = 'C:/Users/lwnpk/Documents/Projects/algtop_cpp_build/bin/Release/debug'
dir_website = "C:/Users/lwnpk/OneDrive/Projects/HTML/WayneLin92.github.io/ss-fb42729d/machine100"

for complex in complexes:
    path_ring = f'{dir_db}/S0_AdamsSS_t100.db'
    path_mod =  f'{dir_db}/{complex}_AdamsSS_t100.db'
    if complex == "S0":
        path_html = f"{dir_website}/S0_AdamsE2_t100/index.html"
        path_js = f"{dir_website}/S0_AdamsE2_t100/data.js"
        data_ring = plot.load_ss_ring(path_ring)
        plot.export_ss_ring(data_ring, path_html, path_js)

        path_html = f"{dir_website}/S0_pi_t100/index.html"
        path_js = f"{dir_website}/S0_pi_t100/data.js"
        data = plot.load_pi_ring(path_ring)
        plot.export_pi_ring(data, path_html, path_js)
    else:
        path_html = f"{dir_website}/{complex}_AdamsE2_t100/index.html"
        path_js = f"{dir_website}/{complex}_AdamsE2_t100/data.js"
        data_ring, data_mod = plot.load_ss_mod(path_ring, path_mod)
        plot.export_ss_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_pi_t100/index.html"
        path_js = f"{dir_website}/{complex}_pi_t100/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_exact_t100/index.html"
        path_js = f"{dir_website}/{complex}_exact_t100/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_exact(data_ring, data_mod, path_html, path_js)

In [19]:
import plot
complexes = ["S0", "C2", "Ceta", "Cnu", "Csigma"]

dir_db = 'C:/Users/lwnpk/Documents/Projects/algtop_cpp_build/bin/Release/main'
dir_website = "C:/Users/lwnpk/OneDrive/Projects/HTML/WayneLin92.github.io/ss-fb42729d/machine"

for complex in complexes:
    path_ring = f'{dir_db}/S0_AdamsSS_t261.db'
    t_trunc = 200 if complex != "C2" else 246
    path_mod =  f'{dir_db}/{complex}_AdamsSS_t{t_trunc}.db'
    if complex == "S0":
        path_html = f"{dir_website}/S0_AdamsE2_t261/index.html"
        path_js = f"{dir_website}/S0_AdamsE2_t261/data.js"
        data_ring = plot.load_ss_ring(path_ring)
        plot.export_ss_ring(data_ring, path_html, path_js)

        path_html = f"{dir_website}/S0_pi_t261/index.html"
        path_js = f"{dir_website}/S0_pi_t261/data.js"
        data = plot.load_pi_ring(path_ring)
        plot.export_pi_ring(data, path_html, path_js)
    else:
        path_html = f"{dir_website}/{complex}_AdamsE2_t{t_trunc}/index.html"
        path_js = f"{dir_website}/{complex}_AdamsE2_t{t_trunc}/data.js"
        data_ring, data_mod = plot.load_ss_mod(path_ring, path_mod)
        plot.export_ss_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_pi_t{t_trunc}/index.html"
        path_js = f"{dir_website}/{complex}_pi_t{t_trunc}/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_exact_t{t_trunc}/index.html"
        path_js = f"{dir_website}/{complex}_exact_t{t_trunc}/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_exact(data_ring, data_mod, path_html, path_js)

In [1]:
import plot
complexes = ["S0", "C2", "Ceta", "Cnu", "Csigma"]

dir_db = 'C:/Users/lwnpk/Documents/Projects/algtop_cpp_build/bin/Release/mix'
dir_website = "C:/Users/lwnpk/OneDrive/Projects/HTML/WayneLin92.github.io/ss-fb42729d/mix"

for complex in complexes:
    path_ring = f'{dir_db}/S0_AdamsSS_t261.db'
    t_trunc = 200 if complex != "C2" else 246
    path_mod =  f'{dir_db}/{complex}_AdamsSS_t{t_trunc}.db'
    if complex == "S0":
        path_html = f"{dir_website}/S0_AdamsE2_t261/index.html"
        path_js = f"{dir_website}/S0_AdamsE2_t261/data.js"
        data_ring = plot.load_ss_ring(path_ring)
        plot.export_ss_ring(data_ring, path_html, path_js)

        path_html = f"{dir_website}/S0_pi_t261/index.html"
        path_js = f"{dir_website}/S0_pi_t261/data.js"
        data = plot.load_pi_ring(path_ring)
        plot.export_pi_ring(data, path_html, path_js)
    else:
        path_html = f"{dir_website}/{complex}_AdamsE2_t{t_trunc}/index.html"
        path_js = f"{dir_website}/{complex}_AdamsE2_t{t_trunc}/data.js"
        data_ring, data_mod = plot.load_ss_mod(path_ring, path_mod)
        plot.export_ss_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_pi_t{t_trunc}/index.html"
        path_js = f"{dir_website}/{complex}_pi_t{t_trunc}/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_mod(data_ring, data_mod, path_html, path_js)

        path_html = f"{dir_website}/{complex}_exact_t{t_trunc}/index.html"
        path_js = f"{dir_website}/{complex}_exact_t{t_trunc}/data.js"
        data_ring, data_mod = plot.load_pi_mod(path_ring,path_mod)
        plot.export_pi_exact(data_ring, data_mod, path_html, path_js)

In [2]:
data_ring = plot.load_ss_ring(path_ring)

In [3]:
data_ring["null_diff_lines"][1]

[(1337, 1399),
 (1367, 1434),
 (1441, 1507),
 (1442, 1507),
 (1662, 1730),
 (1800, 1872),
 (1913, 1998),
 (2213, 2294),
 (2216, 2295),
 (2254, 2365),
 (2299, 2413),
 (2300, 2415),
 (2313, 2429),
 (2365, 2466),
 (2429, 2562),
 (2437, 2572),
 (2488, 2622),
 (2493, 2627),
 (2556, 2673),
 (2569, 2684),
 (2570, 2684),
 (2574, 2697),
 (2626, 2777),
 (2627, 2777),
 (2677, 2835),
 (2678, 2835),
 (2680, 2836),
 (2687, 2841),
 (2688, 2841),
 (2697, 2848),
 (2702, 2853),
 (2703, 2853),
 (2704, 2853),
 (2708, 2858),
 (2771, 2905),
 (2774, 2907),
 (2792, 2920),
 (2793, 2920),
 (2857, 3012),
 (2858, 3012),
 (2862, 3016),
 (2866, 3025),
 (2940, 3092),
 (2994, 3136),
 (3029, 3168),
 (3033, 3178),
 (3038, 3188),
 (3092, 3264),
 (3102, 3274),
 (3136, 3306),
 (3148, 3316),
 (3160, 3323),
 (3161, 3323),
 (3162, 3323),
 (3170, 3332),
 (3171, 3332),
 (3172, 3332),
 (3173, 3332),
 (3174, 3332),
 (3178, 3333),
 (3181, 3336),
 (3182, 3336),
 (3183, 3336),
 (3184, 3336),
 (3190, 3344),
 (3191, 3344),
 (3192, 33